In [1]:
#교차검증 
# 1. train data set 훈련
# 2. validatione data set 평가
# 3. hypter parameter 조정
# 4. train + validation 훈련
# 5. test data 검증

import pandas as pd

wine = pd.read_csv('http://bit.ly/wine_csv_data')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [2]:
# ㅊclass 만 타겟, 나머지는 feature

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine[['class']].to_numpy()

In [3]:
# train set과 test set으로 분리
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2,random_state=42)

In [4]:
#훈련 세트와 검증세트로 구분 현재 80%가 훈련세트
# val input을 훈련에서 20%로 지정함
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=4)

In [5]:
# 훈련셋들 확인
print(sub_input.shape,val_input.shape, test_input.shape)

(4157, 3) (1040, 3) (1300, 3)


In [6]:
#평가하기
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))
# 훈련은 제대로 이루어졌지만 검증값이 제대로 나오지 않았음

0.9968727447678615
0.8692307692307693


In [7]:
#교차검증하기 cross_validation()

from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)
#검증세트를 떼어내지 않고 훈련 세트 전체를 사용함. 
#scores는 fit_time, score_time, test_score 키를 가진 딕셔너리를 반환
#array값이 5개면 5-fold 검사방법

{'fit_time': array([0.00396633, 0.00399947, 0.00399971]), 'score_time': array([0.00099874, 0.00100207, 0.00100112]), 'test_score': array([0.84535488, 0.87182448, 0.84757506])}


C:\Users\gunny\AppData\Local\conda\conda\envs\keras_gpu\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [8]:
#교차검증의 최종 점수는 test_score에 담긴 개수의 평균을 구하는 것. 여기는 5개니 5개의 평균
import numpy as np
print(np.mean(scores['test_score']))

0.8549181380146386


In [9]:
#교차검증을 할대 훈련 세트를 섞으려면 분할기 splitter를 지정해야 함. 
#회귀 모델일 경우 KFold 분할기를 , 분류 모델일 경우 StratifieldKFold를 사용함

from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())

print(np.mean(scores['test_score']))

0.8549181380146386


C:\Users\gunny\AppData\Local\conda\conda\envs\keras_gpu\lib\site-packages\sklearn\model_selection\_split.py:629: FutureWarning: The default value of n_split will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(NSPLIT_WARNING, FutureWarning)


In [10]:
#splitter 분할기를 하용하여 10-KFold 할경우
#n_splits=1 몇 K폴드할지를 정함
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))


0.8626084523194351


In [15]:
#하이퍼파라미터 튜닝
from sklearn.model_selection import GridSearchCV

#매개변수와 탐색할 값을 딕셔너리로 생성
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [16]:
#GridSearchCV: 하이퍼파라미터 찾기 클래스에 탐색 대상 모델과 params 변수를 전달하여 그리드 서치 객체를 만듦
#cv 매개변수가 기본적으로 5여서 5fold 교차 검증함. 즉, params가 5개니 25번을 훈련함. n_jobs는 cpu 코어수 -1은 모든 코어를 사용함
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)


In [17]:
gs.fit(train_input, train_target)

C:\Users\gunny\AppData\Local\conda\conda\envs\keras_gpu\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=42,
                                              splitter='best'),
             iid='warn', n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0

In [18]:
# 25개의 모델 중에서 가장 검증 점수가 가장 높은 모델의 매개변수 조합은 best_estiator 속성에 저장되어 있음
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.8934000384837406


In [20]:
#최적의 매개변수는 best_params_속성에 있음
print(gs.best_params_)

{'min_impurity_decrease': 0.0004}


In [22]:
#평균점수는 mean_test_score에 있음
print(gs.cv_results_['mean_test_score'])

[0.86530691 0.86742351 0.86973254 0.87050221 0.86819319]


In [23]:
#이중에서 고르기 보다 자동으로 numpy argmax()함수를 사용하여 가장 큰 값의 인덱스를 추출할 수 있음
#이 인덱스를 통해 params키에 저장된 매개변수 추출
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0004}


In [25]:
#조금더 많은 하이퍼파라미터 지정
#매개변수와 탐색할 값을 딕셔너리로 생성
#다만 np.arange에서 증감 값을 0.0001로 설정하였으나 그 간격을 둔 것은 특별한 근거가 없음
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
         'max_depth' : range(5, 20, 1),
         'min_samples_split' : range(2, 100, 10)
         }

In [28]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=42,
                                              splitter='best'),
             iid='warn', n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.

In [29]:
#파라미터가 가장 좋은 것 구하기
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 2}


In [30]:
#score 구하기
print(np.max(gs.cv_results_['mean_test_score']))

0.8705022128150857


In [31]:
#scipy (계산용 라이브러리)를 통해서 확률분포 클래스를 통한 랜덤 서치를 진행할 예정
from scipy.stats import uniform, randint

In [35]:
#사용의 예
rgen = randint(0,10)
print(rgen)
rgen.rvs(10)

array([6, 1, 2, 6, 2, 5, 6, 6, 4, 3])

In [36]:
#랜덤으로 생성된 값을 확인하기
# .rvs(갯수)는 rgen을 개수만큼 생성한다는 의미이고, return_counts는 같은 값을 세서 그 숫자를 리턴
# 즉, 이렇게 하더라도 일정한 비율로 생성되는 것을 알 수 있음
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 96,  92, 106,  90,  99, 113, 101,  96, 104, 103], dtype=int64))

In [38]:
# unique 실험 : 실수 생성 : 난수 발생기와 비슷한 개념임
ugen = uniform(0,1)
ugen.rvs(10)

array([0.15566375, 0.7974463 , 0.58865676, 0.61400109, 0.49749804,
       0.94169875, 0.8732095 , 0.54685124, 0.91483206, 0.25947527])

In [40]:
print(uniform(0.0001, 0.001))

In [42]:
#매게변수 생성 : 랜덤으로 생성한 것을 넣기 위해서
# 단 .rvs를 하지 않아도 이렇게 범위만 지정해 주면 됨
params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
         'max_depth' : randint(20, 50),
         'min_samples_split' : randint(2,25),
          'min_samples_leaf' : randint(1,25),
         }

In [43]:
# 랜덤을 위해서는 RandomizedSearchCV를 사용해야 함. n_iter를 지정
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)

gs.fit(train_input, train_target)

C:\Users\gunny\AppData\Local\conda\conda\envs\keras_gpu\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
                   estimator=DecisionTreeClassifier(class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort=False,
                                                    random_state=42,
                                                    splitter='bes

In [44]:
#결과 params 보기
print(gs.best_params_)

{'max_depth': 41, 'min_impurity_decrease': 0.0004586467812961639, 'min_samples_leaf': 10, 'min_samples_split': 3}


In [46]:
#결과 보기
print(np.max(gs.cv_results_['mean_test_score']))

0.8714643063305754


In [47]:
#실제 결과 보기
#훈련된 결과가 best_estimator에 저장
dt = gs.best_estimator_

print(dt.score(test_input, test_target))


0.86
